<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/squad_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# looking at the squad dataset
# getting some of the imports
import requests
import json

In [3]:
files = ['train-v2.0.json', 'dev-v2.0.json']

In [8]:
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/"

In [17]:
# now getting the files 
for file in files:
  res = requests.get(url + file)
  if res.status_code == 200:
    
    with open(file, mode="wb") as f:
     # writing as a chunks 
     for chunk in res.iter_content(chunk_size=50):
       f.write(chunk)



Building one of the files
Building one of the files


In [19]:
# reading in one of the files to be used with json
with open("dev-v2.0.json", mode="rb")as f:
  dev = json.load(f)

with open("train-v2.0.json", mode="rb")as f:
  train = json.load(f)

In [22]:
type(train)

dict

list